<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Tutorials/Customization/tf.function/TensorFlo0w_2_0_Customization_Better_performance_with_tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing TensorFlow

In [1]:
!pip install --upgrade tensorflow-gpu

     |████████████████████████████████| 380.8MB 77kB/s 
     |████████████████████████████████| 450kB 43.1MB/s 
     |████████████████████████████████| 3.8MB 31.5MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


### Importing the dependencies

In [2]:
%%time 
from __future__ import absolute_import, print_function, division, unicode_literals

%load_ext tensorboard
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = "retina"
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import warnings; warnings.simplefilter("ignore")

2.0.0
CPU times: user 1.25 s, sys: 153 ms, total: 1.4 s
Wall time: 1.41 s


### Setting up TensorFlow

In [4]:
tf.debugging.set_log_device_placement(True)
tf.config.set_soft_device_placement(True)
print(f"Executing eagerly: {tf.executing_eagerly()}")
print(f"Intializing random seed {tf.random.set_seed(1)}")
try:
    %tensorflow_version 2.x
except:
    print("Tensorflow version 2 not loaded.")

Executing eagerly: True
Intializing random seed None
TensorFlow is already loaded. Please restart the runtime to change versions.


### Helper code to demostrate different kinds of errors

In [0]:
import contextlib

@contextlib.contextmanager
def assert_raises(error_class):
  try:
    yield
  except error_class as e:
    print('Caught expected exception \n  {}: {}'.format(error_class, e))
  except Exception as e:
    print('Got unexpected exception \n  {}: {}'.format(type(e), e))
  else:
    raise Exception('Expected {} to be raised but no error was raised!'.format(
        error_class))

In [6]:
# a function is like an op
@tf.function
def add(a,b):
    return a+b

add(tf.ones([2,2]), tf.ones([2,2]))

Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_add_13 in device /job:localhost/replica:0/task:0/device:GPU:0


<tf.Tensor: id=14, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [7]:
# Functions have gradients
@tf.function
def add(a,b):
    return a+b

v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __forward_add_34 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference___backward_add_30_35 in device /job:localhost/replica:0/task:0/device:GPU:0


<tf.Tensor: id=38, shape=(), dtype=float32, numpy=1.0>

In [8]:
# can use functions inside functions
@tf.function 
def dense_layer(x,w,b):
    return add(tf.matmul(x,w),b)

dense_layer(tf.ones([3,2]), tf.ones([2,2]), tf.ones([2]))

Executing op __inference_dense_layer_63 in device /job:localhost/replica:0/task:0/device:GPU:0


<tf.Tensor: id=64, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

### Tracing and polymorphism

In [9]:
# functions are polymorphic
@tf.function
def double(a):
    print("Tracing with",a)
    return a+a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("AI")))

Tracing with Tensor("a:0", shape=(), dtype=int32)
Executing op __inference_double_71 in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
Executing op __inference_double_78 in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
Executing op __inference_double_85 in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(b'AIAI', shape=(), dtype=string)


> To control the tracing behaviour, use the following techniques:
    
    * Create a new tf.function. Separate tf.function objects are guarenteed not to share traces.

    * Use `get_concrete_function` method to get a specific trace

    * Specify `input_signature` when calling tf.function to trace only once per calling graph.

In [12]:
print("obtating concrete traces")
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None,dtype=tf.string))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))
print("Using a concrete trace with incompatible types will throw an error")
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1.))

obtating concrete traces
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)
Using a concrete trace with incompatible types will throw an error
Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_91 as input #0(zero-based) was expected to be a string tensor but is a float tensor [Op:__inference_double_91]


In [16]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
  print("Tracing with", x)
  return tf.where(x % 2 == 0, x // 2, 3 * x + 1)
print(next_collatz(tf.constant([1, 2])))
with assert_raises(ValueError):
  next_collatz(tf.constant([[1, 2], [3, 4]]))

Tracing with Tensor("x:0", shape=(None,), dtype=int32)
Executing op __inference_next_collatz_136 in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor([4 1], shape=(2,), dtype=int32)
Caught expected exception 
  <class 'ValueError'>: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


#### When to retrace?

A polymorphic tf.function keeps a cache of concrete functions generated by tracing. The cache keys are effectively tuples of keys generated from the function args and kwargs. The key generated for a tf.Tensor argument is its shape and type. The key generated for a Python primitive is its value. For all other Python types, the keys are based on the object id() so that methods are traced independently for each instance of a class. In the future, TensorFlow may add more sophisticated caching for Python objects that can be safely converted to tensors.